In [2]:
from os import path, getcwd
from IPython.display import Markdown as md
import pandas as pd
import numpy as np
import sqlalchemy as sa
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy.integrate import quad
from scipy.stats import rv_continuous, norm

strYearDBPath=path.join('..','DB', 'year.sqlite3') # путь к рабочей базе данных фактических рядов SQLite 
strExYearDBPath=path.join('..','DB', 'exog_year.sqlite3') # путь к рабочей базе данных экзогенных переменных SQLite
strExParamDBPath=path.join('..','DB', 'exog_param.sqlite3') # путь к рабочей базе данных экзогенных параметров SQLite
strSvodDBPath=path.join('..','DB', 'svod.sqlite3') # путь к рабочей базе данных итоговых рядов SQLite 

# константы базы данных SQLite3
strDatas_table='datas' # название таблицы с данными в базе данных SQLite
strHeaders_table='headers' # название таблицы с текстовыми и кодовыми именами рядов в базе данных SQLite


In [3]:
pd.set_option('display.max_columns',None)

Все данные выбираются из 3 основных источников: БД фактических значений, БД экзогенных значений, БД экзогенных параметров

Экзогенные значения являются прдолжением фактических рядов. Экзогенные параметры задают ряд на всем временном промежутке.

4ый (временный) источник - СВОД. На данном этапе создан отдельный файл (по образу остальных 3ех источников) в который внесены данные с листа "СВОД" краткой версии модели. 
Этот лист является "увязывающим" для всех блоков и в дальнейшем источник СВОД будет ликвидирован, а данные будут поступать напрямую из остальных блоков.

Ряды, получаемые из внешних моделей (вероятности банкротства застройщиков, выбытия жилищного фонда, счет семей улучшивших жилищные условия), пока могут задаваться как экзогенные, либо вручную в коде. В дальнейшем для этого также предполагается использование отдельной БД содержащих результаты расчетов внешних моделей.

На начальном этапе вводятся следующие списки переменных, загружаемых данных: 
<li>list_Year - переменные с листа "параметры", по которым загружаются фактические данные
<li>list_ExYear - переменные с листа "параметры", по которым загружаются экзогенные данные
<li>list_ExParam - переменные с листа "параметры", по которым загружаются экзогенно-задаваемые параметры
<br><br>
Остальные списки переменных содержат переменные, необходимые только для данного блока: "Жилищное строительство"
<li>list_YearZS - переменные, содержащие фактические данные, используемые в блоке "Жилищное строительство". (вынесены на лист "параметры" краткой эксель-версии, т.к. предполагается использование общего датафрейма для всех вводимых в модель переменных)
<li>list_ZSDop - переменные раздела "параметры данного блока", по которым загружаются фактические и экзогенные (в дальнейшем будут подгружаться с помощью list_ZSExDop) данные
<li>list_ZSExParamDop - переменные раздела "параметры данного блока", по которым загружаются экзогенные параметры
<br><br>
Список для перемнных из СВОДа:
<li>list_ZSSvod - переменные раздела "параметры из других блоков моделей", по которым загружаются экзогенные данные

In [4]:
# выбрать переменные по кодам, которые должны использоваться в текущем блоке

# общие для всех блоков фактические ряды
list_Year=('GDP_x','GDP_Iq','CPIAv','KeyRate','Unmpl_s','Inc_x','DispInc_I','ZPN_I',
           'ZPR_I','Invest_Iq','Retail_Iq','Activ_x','ER','Urals','Pop_x','HHAv')
# фактические данные для зависимых переменных (и не только) из блока "ЖИЛИЩНОЕ СТРОИТЕЛЬСТВО"
list_YearZS=('BldProcMKD','VvodyMKD_private','ProjectsPrivate_x','VvodyIZD','VvodySocFB',
             'VvodySocRB','AvSqVv','Vvodyunits','VvodyIZDunits','ZhilFond','ZhilFondIZD',
             'ZhilFondunits','ZhilFondIZDunits','VvodyMKD','MEPop', 'DispPop', 'LivMin')
# общие для всех блоков экзогенные переменные - ???
list_ExYear=('GDP_x','GDP_Iq','CPIAv','KeyRate','Unmpl_s','Inc_x','DispInc_I','ZPN_I',
             'ZPR_I','Invest_Iq','Retail_Iq','Activ_x','ER','Urals','Pop_x')
# общие для всех блоков экзогенные параметры
list_ExParam=('p_ProbDef','p_LabProdOnCost','p_GovPolOnCost','ConstrPriceIZD_x',
              'p_MortgLifeAv_x','HHAv')


# дополнительные данные "Жилищное строительство":
# фактические
list_ZSDop=('Soldaty_x','Badzhilfond_x','queue_x','zhiloutgor_x','zhiloutselo_x')

# экзогенные переменные (Пока выбытия берутся из EXOG, list_ZSExDop совпадает с list_ZSDop)
# list_ZSExDop=('Soldaty_x','Badzhilfond_x','queue_x') # в дальнейшем будет использоваться эта строка

# экзогенные параметры
list_ZSExParamDop=('p_VvodyAll_x','p_VvodyMKD_x','p_sbernas_s','p_sq1room_x','p_sq3room_x',
                   'p_monvmkd_s','p_arendavmon_s','roa_s','yieldofz_s','premium_s',
                   'yieldarenda_s','nowplot_x','goalplot_x','goalplotprom_x',
                   'goalplotnew_x','shareprom_s','potenprom_x','plotprom_i')


# дополнительные данные со Свода (пока только для блока "Жилищное строительство"):
list_ZSSvod=('seb1m_x','price1mddy_x','price1mperv_x')

In [5]:
# общие
strSELECT_Year='''select datas.date, datas.value, headers.code2
                  from {datas_table} join {header_table}
                  on {datas_table}.code={header_table}.code
                  where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_Year)
strSELECT_YearZS='''select datas.date,datas.value,headers.code2 
                    from {datas_table} join {header_table} 
                    on {datas_table}.code={header_table}.code 
                    where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_YearZS)
strSELECT_ExYear='''select datas.date, datas.value, headers.code2 
                    from {datas_table} join {header_table} 
                    on {datas_table}.code={header_table}.code
                    where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ExYear)
strSELECT_ExParam='''select datas.date,datas.value,headers.code2 
                     from {datas_table} join {header_table} 
                     on {datas_table}.code={header_table}.code 
                     where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ExParam)

# для ЖС
strSELECT_ZSDop='''select datas.date, datas.value, headers.code2
                     from {datas_table} join {header_table}
                     on {datas_table}.code={header_table}.code
                     where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ZSDop)
strSELECT_ZSExParamDop='''select datas.date,datas.value,headers.code2 
                        from {datas_table} join {header_table} 
                        on {datas_table}.code={header_table}.code 
                        where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ZSExParamDop)


# из Свод для ЖС
strSELECT_ZSSvod='''select datas.date, datas.value, headers.code2
                     from {datas_table} join {header_table}
                     on {datas_table}.code={header_table}.code
                     where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ZSSvod)

# подключение к базам данных
conWorkDB=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strYearDBPath)) # connection к рабочей базе данных фактических рядов 
conWorkEx=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strExYearDBPath)) # connection к рабочей базе данных экзогенных переменных
conWorkExParam=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strExParamDBPath)) # connection к рабочей базе данных экзогенных параметров
conWorkSvod=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strSvodDBPath)) # connection к рабочей базе данных СВОД


# загрузка данных в датафреймы
pdfYear=pd.read_sql(strSELECT_Year, con=conWorkDB)
pdfYearZS=pd.read_sql(strSELECT_YearZS, con=conWorkDB)
pdfExYear=pd.read_sql(strSELECT_ExYear, con=conWorkEx)
pdfExParam=pd.read_sql(strSELECT_ExParam, con=conWorkExParam)

pdfZSDop=pd.read_sql(strSELECT_ZSDop,con=conWorkDB)
pdfZSExDop=pd.read_sql(strSELECT_ZSDop,con=conWorkEx)
pdfZSExParamDop=pd.read_sql(strSELECT_ZSExParamDop,con=conWorkExParam)

pdfZSSvodIN=pd.read_sql(strSELECT_ZSSvod,con=conWorkSvod)

### Данные с листа "Параметры" (Общие для всех блоков)
выводятся в итоговый датафрейм pdfW

In [6]:
idx=list(range(pdfYear['date'].min(),pdfYear['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfYearMain=pd.DataFrame(index=idx)
for i in pdfYear['code2'].unique():
    pdfYearMain[i]=pdfYear[pdfYear['code2']==i].set_index('date').drop(columns=['code2'])

pdfYearMain[['GDP_x','Inc_x']]*=1000
pdfYearMain[['GDP_Iq','CPIAv','KeyRate','Unmpl_s','DispInc_I','ZPN_I','ZPR_I','Invest_Iq','Retail_Iq']]*=100
pdfYearMain['Activ_x']/=1000
# pdfYearMain

In [7]:
idx=list(range(pdfExYear['date'].min(),pdfExYear['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfExYearMain=pd.DataFrame(index=idx)
for i in pdfExYear['code2'].unique():
    pdfExYearMain[i]=pdfExYear[pdfExYear['code2']==i].set_index('date').drop(columns=['code2'])
pdfExYearMain[['GDP_Iq','CPIAv','Unmpl_s','DispInc_I','ZPN_I','ZPR_I','Invest_Iq','Retail_Iq']]*=100
# pdfExYearMain

In [8]:
idx=list(range(pdfExParam['date'].min(),pdfExParam['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfExParamMain=pd.DataFrame(index=idx)
for i in pdfExParam['code2'].unique():
    pdfExParamMain[i]=pdfExParam[pdfExParam['code2']==i].set_index('date').drop(columns=['code2'])
# pdfExParamMain

In [9]:
pdfW=pdfYearMain.combine_first(pdfExYearMain).combine_first(pdfExParamMain)
# pdfW

In [10]:
# другой способ сделать тоже самое, что и в 3-х ячейках выше

def make_frame(pdf):
    k=pdf.set_index(['date', 'code2']).unstack().reset_index().set_index('date')
    k.columns=[c[1] for c in k.columns]
    return k

def scale(pdf, list_fields=[], multiplier=1):
    pdf[list_fields] *= multiplier
    return pdf

_lst_mult100=['GDP_Iq','CPIAv', 'Unmpl_s','DispInc_I', 'ZPN_I','ZPR_I','Invest_Iq','Retail_Iq']

pdfYearMain_n=(pdfYear.pipe(make_frame)
    .pipe(scale, list_fields=['GDP_x','Inc_x'],         multiplier=1000)
    .pipe(scale, list_fields=_lst_mult100+['KeyRate',], multiplier=100)
    .pipe(scale, list_fields=['Activ_x',],              multiplier=1e-3))

pdfExYearMain_n=pdfExYear.pipe(make_frame).pipe(scale, list_fields=_lst_mult100, multiplier=100)

pdfExParamMain_n=pdfExParam.pipe(make_frame)

pdfW_n=pdfYearMain_n.combine_first(pdfExYearMain_n).combine_first(pdfExParamMain_n)
# pdfW_n

# несовпадения с твоим фреймом. Видимо, там округление где-то сильно на грани точности
pdfW_n.compare(pdfW)

Activ_x           
           self      other
2007  20.125188  20.125188
2008  28.022340  28.022340
2015  82.999700  82.999700

### Данные с листа "Параметры" (фактические ряды для блока "Жилищное строительство")
выводятся в итоговый датафрейм pdfYearZSMain

In [11]:
idx=list(range(pdfYearZS['date'].min(),pdfYearZS['date'].max()+1)) # задается период от первой до последней даты в БД (2004 - 2019 гг.)
pdfYearZSMain=pd.DataFrame(index=idx)
for i in pdfYearZS['code2'].unique():
    pdfYearZSMain[i]=pdfYearZS[pdfYearZS['code2']==i].set_index('date').drop(columns=['code2'])
pdfYearZSMain[['VvodySocFB','VvodySocRB']]*=1000
pdfYearZSMain['LivMin']/=1000
pdfYearZSMain['AvSqIZD']=pdfYearZSMain['VvodyIZD']/pdfYearZSMain['VvodyIZDunits']
# pdfYearZSMain

### Дальше данные с листа "Жилищное строительство"

##### переработанные ряды общих для всех блоков данных с листа "Параметры"
pdfZSList

In [12]:
pdfZSList=pdfW[['GDP_Iq','CPIAv','Unmpl_s','DispInc_I','Pop_x','HHAv']].rename(columns={'GDP_Iq':'GDP_rIq'})
pdfZSList[['CPIAv','Unmpl_s','DispInc_I']]/=100
pdfZSList['GDP_rIq']-=100 
# pdfZSList

##### дополнительные параметры (включая данные из СВОДа)
pdfWDop

In [13]:
idx=list(range(pdfZSDop['date'].min(),pdfZSDop['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfZSDopM=pd.DataFrame(index=idx)
for i in pdfZSDop['code2'].unique():
    pdfZSDopM[i]=pdfZSDop[pdfZSDop['code2']==i].set_index('date').drop(columns=['code2'])
pdfZSDopM['Soldaty_x']*=1000000
pdfZSDopM['queue_x']*=1000

pdfZSDopM.loc[2019,['zhiloutgor_x','zhiloutselo_x']]=np.nan
# pdfZSDopM

In [14]:
idx=list(range(pdfZSExDop['date'].min(),pdfZSExDop['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfZSExDopM=pd.DataFrame(index=idx)
for i in pdfZSExDop['code2'].unique():
    pdfZSExDopM[i]=pdfZSExDop[pdfZSExDop['code2']==i].set_index('date').drop(columns=['code2'])
pdfZSExDopM['Soldaty_x']*=1000000
# pdfZSExDopM

In [15]:
idx=list(range(pdfZSExParamDop['date'].min(),pdfZSExParamDop['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfZSExParamDopM=pd.DataFrame(index=idx)
for i in pdfZSExParamDop['code2'].unique():
    pdfZSExParamDopM[i]=pdfZSExParamDop[pdfZSExParamDop['code2']==i].set_index('date').drop(columns=['code2'])
# pdfZSExParamDopM

In [16]:
idx=list(range(pdfZSSvodIN['date'].min(),pdfZSSvodIN['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfZSSvodINM=pd.DataFrame(index=idx)
for i in pdfZSSvodIN['code2'].unique():
    pdfZSSvodINM[i]=pdfZSSvodIN[pdfZSSvodIN['code2']==i].set_index('date').drop(columns=['code2'])
# pdfZSSvodINM

In [17]:
pdfWDop=pdfZSList.combine_first(pdfZSDopM).combine_first(pdfZSExDopM).combine_first(pdfZSExParamDopM).combine_first(pdfZSSvodINM)
# pdfWDop

<div style="background:beige"><h5> Определение основных дат</h5>

  - `iLastFactYEAR` - последний год фактических данных. 2019 (Последняя точка в БД фактических рядов)
  - `iLastFORECAST` - год горизонта прогноза. 2035 (Последняя точка в БД экзогенных рядов)

</div>

In [18]:
iLastFactYEAR=conWorkDB.connect().execute('select max(date) from datas').fetchone()[0] # pdfYear['date'].max()
iLastFORECAST=conWorkEx.connect().execute('select max(date) from datas').fetchone()[0] # pdfW.index.max()
print('Последний фактический год - ', iLastFactYEAR, '; горизонт прогноза - ', iLastFORECAST)

Последний фактический год -  2019 ; горизонт прогноза -  2035


### Модель 1. Запас жилья в строящихся МКД, на конец года (без учета строящихся за счет средств бюджетов)

Запас жилья в строящихся МКД получается путем сложения значения тренда и отклонения от тренда, рассчитанного по модели.
По фактическим данным запаса жилья в МКД рассчитывается фактический тренд, как скользящее среднее за 2 предшествующих года. Далее находим отклонение от тренда, которое будет зависимой переменной в оцениваемой модели.

In [19]:
## подготовка данных для модели
# Факторы
dfXX=pdfWDop[['seb1m_x','price1mddy_x','GDP_rIq']].copy()

# dfXX['GDP_rIq_1']=dfXX['GDP_rIq'].shift(1)


# уровень маржи, %
dfXX['Margin']=dfXX['price1mddy_x']/dfXX['seb1m_x']*100
# dfXX['Margin_1']=dfXX['Margin'].shift(1)


# прирост маржи за два года, проц. п.
# dfXX['Margin_rI']=dfXX['Margin'].diff(periods=2) 
# вместо цикла ниже

for i in dfXX.index.values[2:]:
    dfXX.loc[i,'Margin_rI']=dfXX.loc[i,'price1mddy_x']/dfXX.loc[i,'seb1m_x']*100-dfXX.loc[i-2,'price1mddy_x']/dfXX.loc[i-2,'seb1m_x']*100

# фиктивная (с убывающим эффектом) переменная адаптации к переходу к новой модели финансирования строительства жилья и карантинным мерам
dfXX['D_NewFinancialModel']=0
dfXX.loc[2019:2021,'D_NewFinancialModel']=[1,1,0.5]


# доля новостроек, недостроенных вследствие вероятного банкротства отдельных застройщиков
dfXX['bnkrpt_s']=0
dfXX.loc[2019:2020,'bnkrpt_s']=[0.150753887354055,0.0477742601266487]


# тренд
# dfXX['TrendF']=pdfYearZSMain['BldProcMKD'].rolling(window=2).mean().shift(1)
# dfXX['dev_from_Trend']=pdfYearZSMain['BldProcMKD']-dfXX['TrendF']
# вместо цикла ниже

for i in pdfYearZSMain.index.values[2:]:
    dfXX.loc[i,'TrendF']=np.mean([pdfYearZSMain.loc[i-2,'BldProcMKD'],pdfYearZSMain.loc[i-1,'BldProcMKD']])
    dfXX['dev_from_Trend']=pdfYearZSMain['BldProcMKD']-dfXX['TrendF']

# dfXX

In [20]:
# Задание горизонта оценивания модели и лагов
iStart1=2011
shiftGDP_rIq=1
shiftMargin=1
shiftMargin_rI=0

<center> $dev\_from\_Trend_i =\beta_1\cdot Margin_{i-1}+\beta_2\cdot Margin\_rI_i+\beta_3\cdot GDP\_rIq_{i-1}+\beta_4\cdot D\_NewFinancialModel + \varepsilon_i$, </center>

где:
- $\beta_1, \beta_2, \beta_3$ и $\beta_4$ - коэффициенты регрессии, $\varepsilon_i$ - случайная ошибка регрессии;
- $dev\_from\_Trend_i$ - отклонение от тренда объема запаса строящегося жилья;
- $Margin_i$ - уровень маржи;
- $Margin\_rI_i$ - прирост маржи за два года;
- $GDP\_rIq_i$ - темп прироста физического объема ВВП;
- $D\_NewFinancialModel$ - дамми на 2019-2021 гг. с убывающим эффектом (адаптация к переходу к новой модели финансирования строительства жилья и карантинным мерам)

In [21]:
mod = smf.ols(formula='''dev_from_Trend ~ Margin.shift({a1}) + Margin_rI.shift({a2}) 
              + GDP_rIq.shift({a3}) + D_NewFinancialModel - 1'''.format(a1=shiftMargin,
                                                                      a2=shiftMargin_rI,
                                                                      a3=shiftGDP_rIq), 
              data=dfXX.loc[iStart1-max(shiftGDP_rIq,shiftMargin,shiftMargin_rI):iLastFactYEAR])
res = mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:         dev_from_Trend   R-squared (uncentered):                   0.964
Model:                            OLS   Adj. R-squared (uncentered):              0.936
Method:                 Least Squares   F-statistic:                              33.65
Date:                Mon, 22 Mar 2021   Prob (F-statistic):                    0.000828
Time:                        20:10:53   Log-Likelihood:                         -19.222
No. Observations:                   9   AIC:                                      46.44
Df Residuals:                       5   BIC:                                      47.23
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

/home/egor/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


После оценивания модели, в каждый прогнозный год рассчитывается значение тренда и суммируется с полученным по модели отклонением от тренда. Тем самым получаем итоговый ряд запасов жилья в строящихся МКД ($BldProcMKD_i$). В 2019 и 2020 гг. значение запасов жилья корректируется на долю новостроек, недостроенных вследствие вероятного банкротства отдельных застройщиков ($bnkrpt\_s_i$).

In [22]:
dfXX['dev_from_TrendM']=res.predict(dfXX)

# вместо всего, что ниже

# def _calc_BldProcMKD(x):
#     return (x['Trend'] + x['dev_from_TrendM']) * (1-x['bnkrpt_s'])

# dfXX['Trend']=dfXX['BldProcMKD'].rolling(2).mean().shift(1)
# dfXX['BldProcMKD']=dfXX.loc[iLastFactYEAR+1:].apply(_calc_BldProcMKD, axis=1).combine_first(pdfYearZSMain['BldProcMKD'])

dfXX['Trend']=dfXX['TrendF']
dfXX['BldProcMKD']=pdfYearZSMain['BldProcMKD']
for i in range(iLastFactYEAR+1,iLastFORECAST+1):
    dfXX.loc[i,'Trend']=np.mean([dfXX.loc[i-2,'BldProcMKD'],dfXX.loc[i-1,'BldProcMKD']])
    dfXX.loc[i,'BldProcMKD']=(dfXX.loc[i,'Trend']+dfXX.loc[i,'dev_from_TrendM'])*(1-dfXX.loc[i,'bnkrpt_s'])
# dfXX

### Модель 2. Баланс объемов стройки, запусков и вводов жилья

In [23]:
dfzs2=pdfYearZSMain[['VvodyMKD_private','ProjectsPrivate_x']].combine_first(dfXX[['BldProcMKD']])
# dfzs2

In [24]:
# dfzs2[['t1', 't2', 't3']]=[0.25, 0.5, 0.25]

dfzs2['p1']=0.25
dfzs2.loc[2021,'p1']=0.35

dfzs2['p2']=0.5
dfzs2.loc[2020:2021,'p2']=[0.4,0.6]

dfzs2['p3']=0.25
dfzs2.loc[2020,'p3']=0.15

# dfzs2

Объемы вводов МКД:
<center> $VvodyMKD\_private =p_1\cdot ProjectsPrivate\_x_{i-3}+p_2\cdot ProjectsPrivate\_x_{i-2}+p_3\cdot ProjectsPrivate\_x_{i-1}$, </center>

где:
- $p_1, p_2$ и $p_3$ - параметры, задающие долю вводов жилья от выходящих на рынок жилищных проектов 3, 2 и 1 год назад, соответственно. На 2020 г. устанавливаются $p_1=0.25, p_2=0.4$ и $p_3=0.15$. На 2021 г.: $p_1=0.35, p_2=0.6$ и $p_3=0.25$. На остальном прогнозном промежутке: $p_1=0.25, p_2=0.5$ и $p_3=0.25$;
- $ProjectsPrivate\_x_i$ - объем выходящих на рынок жилищных проектов за вычетом строящихся за счет средств государственного бюджета

Объем выходящих на рынок жилищных проектов:
<center> $ProjectsPrivate\_x_i =BldProcMKD_i-BldProcMKD_{i-1}+VvodyMKD\_private_i$, </center>

где:
- $BldProcMKD_i$ - запас жилья в строящихся МКД, на конец года i за вычетом строящихся за счет средств государственного бюджета;
- $VvodyMKD\_private_i$ - объем вводов МКД за вычетом строящихся за счет средств государственного бюджета;

In [25]:
# dfzs2.loc[iLastFactYEAR,'ProjectsPrivate_x']=dfzs2.loc[iLastFactYEAR,'BldProcMKD']-dfzs2.loc[iLastFactYEAR-1,'BldProcMKD']+dfzs2.loc[iLastFactYEAR,'VvodyMKD_private']
for i in dfzs2.loc[iLastFactYEAR:].index:
    if pd.isna(dfzs2.loc[i,'VvodyMKD_private']):
        
        # после равно - вычисления в скобках - длая переноса строк кода, чтобы лучше видно было
        dfzs2.loc[i,'VvodyMKD_private']=(dfzs2.loc[i-3,'ProjectsPrivate_x']*dfzs2.loc[i,'p1'] + 
        dfzs2.loc[i-2,'ProjectsPrivate_x']*dfzs2.loc[i,'p2'] +
        dfzs2.loc[i-1,'ProjectsPrivate_x']*dfzs2.loc[i,'p3'])
        
    dfzs2.loc[i,'ProjectsPrivate_x']=dfzs2.loc[i,'BldProcMKD']-dfzs2.loc[i-1,'BldProcMKD']+dfzs2.loc[i,'VvodyMKD_private']
# dfzs2

In [26]:
# тут  такое дело:
# правильнее и быстрее делать через .iterrows() - эта штука возвращает итератор по датафрейму в виде индекс, строка (ряд) данных как серия

# а вот с векторами (матричное перемножение) хоть и правильно, но совершенно непонятно стало, что на что умножается
# насчет выигрыша по времени выполнения - нууу, не знаю. Не уверен. Фрейм маленький, выигрыш незаметный, зато понятность потерялась

# так что видимо у тебя вариант лучше, только я бы все же на iterrows() переделал

# насчет pipe - это просто удобная форма. В даном случае не обязательно, а вот вообще pipe принимает фреймы, группировки и серии, 
# то есть можносерьезные огороды городить 

# но у тебя вариант понятнее - только iterrows(). Там почти все локи по фрейму уходят

def _calc_VvodyMKD(pdf):
    for i, val in pdf.loc[iLastFactYEAR:].iterrows():

        if pd.isna(val['VvodyMKD_private']):
            x_row=val['p1':'p3'].to_numpy()
            x_col=np.atleast_2d(pdf.loc[i-3:i-1, 'ProjectsPrivate_x']).T
            val['VvodyMKD_private']= (x_row @ x_col)[0]
            
        val['ProjectsPrivate_x']=val['BldProcMKD']-pdf.loc[i-1,'BldProcMKD']+val['VvodyMKD_private']
    return pdf
    
dfzs2.pipe(_calc_VvodyMKD)

,BldProcMKD,ProjectsPrivate_x,VvodyMKD_private,p1,p2,p3
2004,59.386684,27.254825,22.167600,0.25,0.5,0.25
2005,66.863359,30.726575,23.249900,0.25,0.5,0.25
2006,70.858939,31.297480,27.301900,0.25,0.5,0.25
2007,68.941974,29.248635,31.165600,0.25,0.5,0.25
2008,64.672504,29.003730,33.273200,0.25,0.5,0.25
2009,68.476586,31.282002,27.477920,0.25,0.5,0.25
2010,73.837691,34.126605,28.765500,0.25,0.5,0.25
2011,81.098644,38.266952,31.006000,0.25,0.5,0.25
2012,90.318934,43.570800,34.350510,0.25,0.5,0.25
2013,99.986909,46.467375,36.799400,0.25,0.5,0.25


### Модель 3. Модель вводов ИЖД + Модель 4. Отклонение от тренда объема ввода ИЖД на одного человека

Вводы ИЖД на прогнозном периоде получаются сложением вводов ИЖД, расчитанных по модели (текущий спрос) и дополнительных вводов в рамках стимулирующей госпрограммы для достижения целевых значений нацпроекта. ($VvodyIZD_i = VvodyIZD\_M_i + DopVvody_i$) 

Для модели вводов ИЖД сначала фактический объем вводов корректируется на объем населения. ($VvodyIZD\_per\_capita_i = VvodyIZD_i / Pop\_x_i$)
Далее рассчитывается тренд объема вводов ИЖД на одного человека, как скользящая средняя за 4 предшествующих года с весами 5% для t-4, 10% для t-3, 25% для t-2 и 60% для t-1. 
Вычитая из фактического значения объемов вводов ИЖД на одного человека значение тренда получаем отклонение от тренда объема ввода ИЖД на одного человека, которое выступает зависимой переменной в оцениваемой модели.

In [27]:
dfzs3=pdfYearZSMain[['VvodyIZD']].combine_first(pdfWDop[['DispInc_I','Unmpl_s']])
dfzs3['VvodyIZD_per_capita']=dfzs3['VvodyIZD']/pdfWDop['Pop_x']


p1=0.05
p2=0.1
p3=0.25
p4=0.6

# def fff(x):
#     return x.iloc[0]*0.05+x.iloc[1]*0.1+x.iloc[2]*0.25+x.iloc[3]*0.6

dfzs3['VvodyIZD_per_capita_MA']=dfzs3['VvodyIZD_per_capita'].rolling(4).apply(lambda x: x.iloc[0]*p1+x.iloc[1]*p2+x.iloc[2]*p3+x.iloc[3]*p4)

dfzs3['dev_from_trend_VvodyIZD']=dfzs3['VvodyIZD_per_capita']-dfzs3['VvodyIZD_per_capita_MA'].shift(1)

dfzs3['d3_DispInc_I']=dfzs3['DispInc_I'].rolling(3).apply(lambda x: np.prod(x)*100-100)
dfzs3['d3_Unmpl_s']=dfzs3['Unmpl_s'].rolling(3).apply(lambda x: (x.iloc[2]-x.iloc[0])*100)

dfzs3['D_IZD']=0
dfzs3.loc[2014,'D_IZD']=1

dfzs3

,DispInc_I,Unmpl_s,VvodyIZD,VvodyIZD_per_capita,VvodyIZD_per_capita_MA,dev_from_trend_VvodyIZD,d3_DispInc_I,d3_Unmpl_s,D_IZD
2004,1.104000,0.077149,16.09290,0.111704,NaN,NaN,NaN,NaN,0
2005,1.124000,0.071532,17.53000,0.122144,NaN,NaN,NaN,NaN,0
2006,1.135000,0.070622,19.96880,0.139594,NaN,NaN,40.841696,-0.652645,0
2007,1.121000,0.060294,26.30710,0.184217,0.163228,NaN,43.010454,-1.123850,0
2008,1.024000,0.062412,27.36700,0.191723,0.181155,0.028495,30.287104,-0.821027,0
2009,1.030000,0.082522,28.54614,0.199923,0.193286,0.018769,18.234112,2.222843,0
2010,1.059000,0.073445,25.51820,0.178637,0.185546,-0.014649,11.694848,1.103351,0
2011,1.005000,0.064956,26.76100,0.187191,0.186552,0.001645,9.622385,-1.756593,0
2012,1.046000,0.054587,28.37760,0.198165,0.193557,0.011613,11.325257,-1.885879,0
2013,1.040000,0.054780,30.66330,0.213671,0.205395,0.020114,9.327920,-1.017603,0


Объем выходящих на рынок жилищных проектов:
<center> $dev\_from\_trend\_VvodyIZD_i= \beta_1 \cdot d3\_DispInc\_I_{i-1}+ \beta_2 \cdot d3\_Unmpl\_s_i+ \beta_3 \cdot D\_IZD + \varepsilon_i$, </center>

где:
- $\beta_1, \beta_2$ и $\beta_3$ - коэффициенты регрессии, $\varepsilon_i$ - случайная ошибка регрессии;
- $d3\_DispInc\_I_{i-1}$ - изменение реальных располагаемых доходов населения за 3 года (лаг = 1 год);
- $d3\_Unmpl\_s_i$ - изменение безработицы за 3 года (в период t по сравнению с периодом t-2);
- $D\_IZD$ - фиктивная переменная: ажиотаж перед завершением "дачной амнистии";

In [28]:
# Задание горизонта оценивания модели и лагов
iStart3=2009
shiftDispInc=1
shiftUnmpl=0

In [29]:
mod4 = smf.ols(formula='''dev_from_trend_VvodyIZD ~ d3_DispInc_I.shift({a1}) + d3_Unmpl_s.shift({a2}) 
              + D_IZD -1'''.format(a1=shiftDispInc,
                                   a2=shiftUnmpl), 
              data=dfzs3.loc[iStart3-max(shiftDispInc,shiftUnmpl):iLastFactYEAR])
res4 = mod4.fit()
print(res4.summary())

                                   OLS Regression Results                                   
Dep. Variable:     dev_from_trend_VvodyIZD   R-squared (uncentered):                   0.451
Model:                                 OLS   Adj. R-squared (uncentered):              0.246
Method:                      Least Squares   F-statistic:                              2.194
Date:                     Mon, 22 Mar 2021   Prob (F-statistic):                       0.167
Time:                             20:10:54   Log-Likelihood:                          29.952
No. Observations:                       11   AIC:                                     -53.90
Df Residuals:                            8   BIC:                                     -52.71
Df Model:                                3                                                  
Covariance Type:                 nonrobust                                                  
                            coef    std err          t      P>|t|     

/home/egor/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


На прогнозном промежутке:

После оценивания модели, в каждый прогнозный год получаем значение объема вводов ИЖД на одного человека путем сложения тренда и отклонения от тренда, рассчитанного по модели, а затем пересчитываем тренд на следующий год. 

Вводы ИЖД по текущему спросу получаются путем домножения объема вводов ИЖД на одного человека на объем населения. А дополнительные вводы в рамках госпрограммы рассчитываются как разница вводов ИЖД запланированных в рамках НП "Жилье и городская среда" (разница совокупных вводов жилья и вводов МКД) и вводов ИЖД по текущему спросу, рассчитанных в модели.

In [30]:
dfzs3['dev_from_trend_VvodyIZD_M']=res4.predict(dfzs3)
for i in range(iLastFactYEAR+1,iLastFORECAST+1):
    dfzs3.loc[i,'VvodyIZD_per_capita']=dfzs3.loc[i-1,'VvodyIZD_per_capita_MA']+dfzs3.loc[i,'dev_from_trend_VvodyIZD_M']
#     dfzs3.loc[i,'VvodyIZD_per_capita_MA']=dfzs3.loc[i-3,'VvodyIZD_per_capita']*p1+dfzs3.loc[i-2,'VvodyIZD_per_capita']*p2+dfzs3.loc[i-1,'VvodyIZD_per_capita']*p3+dfzs3.loc[i,'VvodyIZD_per_capita']*p4
    dfzs3.loc[i,'VvodyIZD_per_capita_MA']=dfzs3.loc[i-3:,'VvodyIZD_per_capita'].rolling(4).apply(lambda x: x.iloc[0]*p1+x.iloc[1]*p2+x.iloc[2]*p3+x.iloc[3]*p4)[i]
dfzs3['VvodyIZD_M']=(dfzs3['VvodyIZD_per_capita_MA'].shift(1)+dfzs3['dev_from_trend_VvodyIZD_M'])*pdfWDop['Pop_x']


for i in range(iLastFactYEAR+1,iLastFORECAST+1):
    if pdfWDop.loc[i,'p_VvodyAll_x']-pdfWDop.loc[i,'p_VvodyMKD_x']>dfzs3.loc[i,'VvodyIZD_M']:
        dfzs3.loc[i,'DopVvody']=pdfWDop.loc[i,'p_VvodyAll_x']-pdfWDop.loc[i,'p_VvodyMKD_x']-dfzs3.loc[i,'VvodyIZD_M']
    dfzs3.loc[i,'VvodyIZD']=dfzs3.loc[i,'VvodyIZD_M']+dfzs3.loc[i,'DopVvody']

# dfzs3

### Модель 5. Модель объема ввода МКД для институциональной аренды

Объем ввода МКД для институциональной аренды рассчитывается как произведение вводов жилья в МКД, доли монолитных домов в вобщем вводе МКД и доли арендного жилья от ввода монолитных МКД.
$VvodyMKD\_inst_i = VvodyMKD\_private_i \cdot p\_monvmkd\_s_i \cdot p\_arendavmon\_s_i$

In [31]:
dfzs5=dfzs2[['VvodyMKD_private']].combine_first(pdfWDop[['p_monvmkd_s','p_arendavmon_s']])
start5=2018
dfzs5['VvodyMKD_inst'] = dfzs5.loc[start5:][list(dfzs5.columns)].prod(axis=1)
dfzs5

,VvodyMKD_private,p_arendavmon_s,p_monvmkd_s,VvodyMKD_inst
2004,22.167600,NaN,NaN,NaN
2005,23.249900,NaN,NaN,NaN
2006,27.301900,NaN,NaN,NaN
2007,31.165600,NaN,NaN,NaN
2008,33.273200,NaN,NaN,NaN
2009,27.477920,NaN,NaN,NaN
2010,28.765500,NaN,NaN,NaN
2011,31.006000,NaN,NaN,NaN
2012,34.350510,NaN,NaN,NaN
2013,36.799400,NaN,NaN,NaN


### Модель 6.1 Модель объема ввода социального жилья, построенного за счет средств федерального бюджета

In [32]:
dfzs6=pdfYearZSMain[['VvodySocFB','VvodySocRB']].combine_first(pdfWDop[['Soldaty_x','Badzhilfond_x','queue_x']])

Объем ввода социального жилья, построенного за счет средств федерального бюджета:
<center> $VvodySocFB_i=\beta_0 + \beta_1 \cdot Soldaty\_x_{i-1} + \varepsilon_i$, </center>

где:
- $\beta_0$ - константа, $\beta_1$ - коэффициент регрессии, $\varepsilon_i$ - случайная ошибка;
- $Soldaty\_x_{i-1}$ - Количество человек, планируемых к увольнению с военной службы (лаг = 1 год);

In [33]:
iStart6_1=2013
iEnd6_1=2017
shiftSoldaty=1

In [34]:
mod6_1 = smf.ols(formula='''VvodySocFB ~ Soldaty_x.shift({a1})'''.format(a1=shiftSoldaty), 
                 data=dfzs6.loc[iStart6_1-shiftSoldaty:iEnd6_1])
res6_1 = mod6_1.fit()
print(res6_1.summary())

                            OLS Regression Results                            
Dep. Variable:             VvodySocFB   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     1.445
Date:                Mon, 22 Mar 2021   Prob (F-statistic):              0.316
Time:                        20:10:55   Log-Likelihood:                -34.114
No. Observations:                   5   AIC:                             72.23
Df Residuals:                       3   BIC:                             71.45
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept            413.7969    605

/home/egor/anaconda3/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [35]:
dfzs6['VvodySocFB_M']=res6_1.predict(dfzs6)
dfzs6.VvodySocFB.fillna(dfzs6.VvodySocFB_M, inplace=True)
dfzs6

,Badzhilfond_x,Soldaty_x,VvodySocFB,VvodySocRB,queue_x,VvodySocFB_M
2004,NaN,NaN,NaN,NaN,NaN,NaN
2005,NaN,NaN,NaN,NaN,NaN,NaN
2006,NaN,NaN,NaN,NaN,NaN,NaN
2007,NaN,NaN,NaN,NaN,NaN,NaN
2008,2.333110,NaN,882.900000,2535.3,2864.000000,NaN
2009,2.910700,NaN,1399.400000,2468.1,2830.000000,NaN
2010,2.584900,NaN,2634.000000,1513.0,2818.000000,NaN
2011,2.387000,NaN,2979.000000,1518.6,2799.000000,NaN
2012,2.043990,489423.000000,1397.400000,1616.0,2748.000000,NaN
2013,2.950710,367067.000000,1282.400000,1739.8,2683.000000,1409.230816


### Модель 6.2 Модель объема ввода социального жилья, построенного за счет средств бюджетов субъектов Российской Федерации и местных бюджетов

Объем  ввода социального жилья, построенного за счет средств бюджетов субъектов Российской Федерации и местных бюджетов:
<center> $VvodySocRB_i=\beta_1 \cdot Badzhilfond\_x_{i+1} + \beta_2 \cdot queue\_x_i + \varepsilon_i$, </center>

где:
- $\beta_1, \beta_2$ - коэффициенты регрессии, $\varepsilon_i$ - случайная ошибка;
- $Badzhilfond\_x_{i+1}$ - объем снесенной общей площади жилищного фонда по ветхости и аварийности (значение показателя за год, опережение = 1 год);
- $queue\_x_i$ - число семей (включая одиноких), состоявших на учете в качестве нуждающихся в жилых помещениях (на конец года);

In [36]:
iStart6_2=2008
iEnd6_2=2014
shiftBadFond=-1
shiftQueue=0

In [37]:
mod6_2 = smf.ols(formula='''VvodySocRB ~ Badzhilfond_x.shift({a1}) + queue_x.shift({a2}) -1'''.format(a1=shiftBadFond,
                                                                                                      a2=shiftQueue), 
                 data=dfzs6.loc[iStart6_2-max(shiftBadFond,shiftQueue):iEnd6_2-min(shiftBadFond,shiftQueue)])
res6_2 = mod6_2.fit()
print(res6_2.summary())

                                 OLS Regression Results                                
Dep. Variable:             VvodySocRB   R-squared (uncentered):                   0.951
Model:                            OLS   Adj. R-squared (uncentered):              0.931
Method:                 Least Squares   F-statistic:                              48.16
Date:                Mon, 22 Mar 2021   Prob (F-statistic):                    0.000541
Time:                        20:10:55   Log-Likelihood:                         -52.151
No. Observations:                   7   AIC:                                      108.3
Df Residuals:                       5   BIC:                                      108.2
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

/home/egor/anaconda3/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 7 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


В 2020 г. делается поправка полученного значения вводов на эффект карантинных мер и экономического кризиса (-1000 тыс. кв. м)

In [38]:
dfzs6['VvodySocRB_M']=res6_2.predict(dfzs6)
dfzs6.loc[2020,'VvodySocRB_M']-=1000 # поправка на эффект карантинных мер и экономического кризиса (экспертно)
dfzs6.VvodySocRB.fillna(dfzs6.VvodySocRB_M, inplace=True)
dfzs6

,Badzhilfond_x,Soldaty_x,VvodySocFB,VvodySocRB,queue_x,VvodySocFB_M,VvodySocRB_M
2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,2.333110,NaN,882.900000,2535.300000,2864.000000,NaN,1913.136473
2009,2.910700,NaN,1399.400000,2468.100000,2830.000000,NaN,1837.479767
2010,2.584900,NaN,2634.000000,1513.000000,2818.000000,NaN,1795.705055
2011,2.387000,NaN,2979.000000,1518.600000,2799.000000,NaN,1724.168420
2012,2.043990,489423.000000,1397.400000,1616.000000,2748.000000,NaN,1864.353661
2013,2.950710,367067.000000,1282.400000,1739.800000,2683.000000,1409.230816,1807.908898


## Датафрейм для результатов (должен создаваться раньше и заполняться по мере поступления информации)

Вводы МКД в государственную собственность (социальное жилье): $VvodyMKD\_gov_i = \dfrac {VvodySocFB_i + VvodySocRB_i}{1000}$

In [39]:
dfzs_res=pdfYearZSMain[['VvodyMKD']].combine_first(dfzs2[['VvodyMKD_private']]).combine_first(dfzs5[['VvodyMKD_inst']]).combine_first(pd.DataFrame((dfzs6['VvodySocFB']+dfzs6['VvodySocRB'])/1000, columns=['VvodyMKD_gov']))
dfzs_res.loc[iLastFactYEAR+1:,'VvodyMKD']=dfzs_res.loc[iLastFactYEAR+1:][['VvodyMKD_gov','VvodyMKD_inst','VvodyMKD_private']].sum(axis=1)
# dfzs_res

### Модель 6.3 Модель объема ввода жилья, предназначенного для некоммерческой аренды

Вводы жилья, предназначенного для некоммерческой аренды $(VvodyNKA)$ рассчитываются как положительная разница между планируемыми вводами жилья в МКД в рамках НП и совокупными вводами МКД, рассчитанными в предыдущих разделах данного блока (сумма вводов МКД в частную - $VvodyMKD\_private$, институциональную - $VvodyMKD\_inst$ и государственную собственность - $VvodyMKD\_gov$). Если разница отрицательная, то вводы жилья для некоммерческой аренды предполагаются нулевыми.

$
\begin{equation*}
VvodyNKA_i =  
 \begin{cases}
   p\_VvodyMKD\_x_i - VvodyMKD_i, &\text{ если } p\_VvodyMKD\_x_i - VvodyMKD_i>0 \\
   0, &\text{ если } p\_VvodyMKD\_x_i - VvodyMKD_i<0
 \end{cases}
\end{equation*}
$, где
- $p\_VvodyMKD\_x_i$ - совокупный ввод жилья в МКД в рамках Национального проекта "Жилье и городская среда"
- Совокупные вводы МКД: $VvodyMKD_i = VvodyMKD\_private_i + VvodyMKD\_inst_i + VvodyMKD\_gov_i$


Необходимые  инвестиции рассчитываются по следующей формуле: 

$InvNKA_i =  VvodyNKA_{i+1} \cdot \dfrac{price1mperv\_x_i}{(1+\dfrac{roa\_s_i}{100})}$, где
- $VvodyNKA_{i+1}$ - вводы жилья, предназначенного для некоммерческой аренды (опережение = 1 год);
- $price1mperv\_x_i$ - средняя цена 1 кв. м на первичном рынке;
- $roa\_s_i$ - рентабельность строительства;

Субсидия со стороны государства (накопленным итогом):
$GovSubsNKA_i = \dfrac{yieldofz\_s_i+premium\_s_i-yieldarenda\_s_i}{100} \cdot \Sigma^{i-1}_{2020}InvNKA$, где

- $yieldofz\_s_i$ - доходность по 30-летним ОФЗ;
- $premium\_s_i$ - премия инвестора;
- $yieldarenda\_s_i$ - ставка аренды для физлица;

In [40]:
dfzs6=dfzs6.combine_first(pdfWDop[['p_VvodyMKD_x','roa_s','yieldofz_s','premium_s','yieldarenda_s','price1mperv_x']]).combine_first(dfzs_res[['VvodyMKD']])
iStart6_3=2021
# dfzs6

In [41]:
for i in range(iStart6_3,iLastFORECAST+1):
    dfzs6.loc[i,'VvodyNKA']=dfzs6.loc[i,'p_VvodyMKD_x']-dfzs6.loc[i,'VvodyMKD']
    if (dfzs6.loc[i,'p_VvodyMKD_x']-dfzs6.loc[i,'VvodyMKD'])<0:
        dfzs6.loc[i,'VvodyNKA']=0
    dfzs6.loc[i-1,'InvNKA']=dfzs6.loc[i,'VvodyNKA']*dfzs6.loc[i,'price1mperv_x']/(1+dfzs6.loc[i,'roa_s']/100)
    dfzs6.loc[i,'GovSubsNKA']=dfzs6.loc[iLastFactYEAR+1:i-1,'InvNKA'].sum(axis=0)*(dfzs6.loc[i,'yieldofz_s']+dfzs6.loc[i,'premium_s']-dfzs6.loc[i,'yieldarenda_s'])/100 

In [42]:
dfzs6

,Badzhilfond_x,Soldaty_x,VvodyMKD,VvodySocFB,VvodySocFB_M,VvodySocRB,VvodySocRB_M,p_VvodyMKD_x,premium_s,price1mperv_x,queue_x,roa_s,yieldarenda_s,yieldofz_s,VvodyNKA,InvNKA,GovSubsNKA
2004,NaN,NaN,24.947200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,NaN,NaN,26.029500,NaN,NaN,NaN,NaN,NaN,NaN,23.771597,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,NaN,NaN,30.583300,NaN,NaN,NaN,NaN,NaN,NaN,31.473560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,NaN,NaN,34.914200,NaN,NaN,NaN,NaN,NaN,NaN,43.883193,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,2.333110,NaN,36.691400,882.900000,NaN,2535.300000,1913.136473,NaN,NaN,51.332988,2864.000000,NaN,NaN,NaN,NaN,NaN,NaN
2009,2.910700,NaN,31.345420,1399.400000,NaN,2468.100000,1837.479767,NaN,NaN,48.865360,2830.000000,NaN,NaN,NaN,NaN,NaN,NaN
2010,2.584900,NaN,32.912500,2634.000000,NaN,1513.000000,1795.705055,NaN,NaN,48.122768,2818.000000,NaN,NaN,NaN,NaN,NaN,NaN
2011,2.387000,NaN,35.503600,2979.000000,NaN,1518.600000,1724.168420,NaN,NaN,42.581430,2799.000000,NaN,NaN,NaN,NaN,NaN,NaN
2012,2.043990,489423.000000,37.363910,1397.400000,NaN,1616.000000,1864.353661,NaN,NaN,46.740097,2748.000000,NaN,NaN,NaN,NaN,NaN,NaN
2013,2.950710,367067.000000,39.821600,1282.400000,1409.230816,1739.800000,1807.908898,NaN,NaN,49.573295,2683.000000,NaN,NaN,NaN,NaN,NaN,NaN


### Модель 7.1 Средняя площадь квартиры в вводимом жилье

In [43]:
dfzs7=pdfYearZSMain[['AvSqVv','MEPop','DispPop','LivMin','AvSqIZD']].combine_first(pdfWDop[['p_sbernas_s','p_sq1room_x','p_sq3room_x','price1mperv_x']]).combine_first(pdfZSList[['CPIAv','Unmpl_s','DispInc_I','HHAv']])

$ \text{Предварительные расчеты }$

Берем фактические значения 
- математического ожидания плотности распределения населения по логарифму от среднедушевых денежных доходов (нормальный закон распределения) ($MEPop$);
- дисперсии плотности распределения населения по логарифму от среднедушевых денежных доходов ($DispPop$);
- величины прожиточного минимума ($LivMin$). 

На прогнозном периоде:

$MEPop_i = MEPop_{i-1} + ln(CPIAv_i \cdot DispInc\_I_i)$ <br>
изменение матожидания раскручивается темпами ИПЦ и реальных располагаемых доходов населения

$DispPop_i=DispPop_{i-1}$ <br>
диспресия предполагается постоянной на уровне последнего фактического значения

$LivMin_i=LivMin_{i-1} \cdot CPIAv_i$ <br>
прожиточный минимум изменяется в соответствии с темпом  роста цен


Далее необходимо рассчитать
- Долю населения, способного приобрести 1-2 комнатные квартиры ($PopAble2_s$);
- Долю населения, способного приобрести 3-х комнатные квартиры ($PopAble3_s$);
- Скользящие средние этих показателей за 3 года.

$PopAble3\_s_i= (1 - x_i)\cdot 100$,

где $ x_i=\phi({ \ln{ ( ({\dfrac{p\_sq3room\_x_i * price1mperv\_x_i}{36*HHAv_i} + LivMin_i*2})*1000 )} }) \sim N(MEPop_i,DispPop_i)$

$PopAble2\_s_i= (1 - x_i)\cdot 100 - PopAble3\_s_i$, 

где $ x_i=\phi({ \ln{ ( ({\dfrac{p\_sq1room\_x_i * price1mperv\_x_i}{36*HHAv_i} + LivMin_i*2})*1000 )} }) \sim N(MEPop_i,DispPop_i)$

In [44]:
for i in range(iLastFactYEAR+1,iLastFORECAST+1):
    dfzs7.loc[i,'MEPop']=dfzs7.loc[i-1,'MEPop']+np.log(dfzs7.loc[i,'CPIAv']*dfzs7.loc[i,'DispInc_I']) # cumsum
    dfzs7.loc[i,'DispPop']=dfzs7.loc[i-1,'DispPop']
    dfzs7.loc[i,'LivMin']=dfzs7.loc[i-1,'LivMin']*dfzs7.loc[i,'CPIAv']                                # cumprod

# DemandOM.pdfWork['p_middleclass']=1-norm.cdf(np.log(DemandOM.pdfWork['_mid_level']), 
#                                               loc=DemandOM.pdfWork['MEPop'], 
#                                               scale=DemandOM.pdfWork['DispPop'])

def norm_people_room(x, sName):
#     _x=np.log( (x['p_sq3room_x']*x['price1mperv_x']/(36*x['HHAv']) + x['LivMin']*2)*1000)
    _x=np.log( (x[sName]*x['price1mperv_x']/(36*x['HHAv']) + x['LivMin']*2)*1000)
    return (1-norm.cdf(_x, loc=x['MEPop'], scale=x['DispPop']))*100

dfzs7['PopAble3_s']=dfzs7.apply(norm_people_room, axis=1, args=('p_sq3room_x', ))
dfzs7['PopAble2_s']=dfzs7.apply(norm_people_room, axis=1, args=('p_sq1room_x', )) - dfzs7['PopAble3_s']

dfzs7['PopAble3_s_MA']=dfzs7['PopAble3_s'].rolling(3).apply(lambda x: np.mean(x))
dfzs7['PopAble2_s_MA']=dfzs7['PopAble2_s'].rolling(3).apply(lambda x: np.mean(x))

In [45]:
dfzs7

,AvSqIZD,AvSqVv,CPIAv,DispInc_I,DispPop,HHAv,LivMin,MEPop,Unmpl_s,p_sbernas_s,p_sq1room_x,p_sq3room_x,price1mperv_x,PopAble3_s,PopAble2_s,PopAble3_s_MA,PopAble2_s_MA
2004,138.731897,68.975069,1.108812,1.104000,NaN,2.677054,2.376000,NaN,0.077149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,138.031496,67.268041,1.127055,1.124000,0.756100,2.661470,3.018000,8.711611,0.071532,0.203668,33.823529,68.823529,23.771597,3.856509,8.765876,NaN,NaN
2006,138.672222,65.806452,1.096830,1.135000,0.769907,2.645886,3.422000,8.928732,0.070622,0.213574,34.588235,70.588235,31.473560,3.593274,8.939490,NaN,NaN
2007,138.458421,65.601504,1.089881,1.121000,0.785663,2.630302,3.847000,9.129498,0.060294,0.217936,35.352941,72.352941,43.883193,2.833931,8.264237,3.427905,8.656534
2008,136.835000,64.612676,1.141048,1.024000,0.782878,2.614717,4.593000,9.301331,0.062412,0.214176,36.117647,74.117647,51.332988,2.683727,8.091246,3.036978,8.431658
2009,135.934000,63.821138,1.116977,1.030000,0.783533,2.599133,5.153000,9.427587,0.082522,0.217221,36.882353,75.882353,48.865360,3.793679,9.635617,3.103779,8.663700
2010,132.907292,62.666667,1.068508,1.059000,0.782280,2.583549,5.688000,9.546797,0.073445,0.214670,37.647059,77.647059,48.122768,4.782465,10.837774,3.753291,9.521546
2011,133.139303,60.683761,1.084634,1.005000,0.775441,2.544156,6.369000,9.644999,0.064956,0.207514,38.411765,79.411765,42.581430,6.601753,12.297760,5.059299,10.923717
2012,134.490995,59.489633,1.050667,1.046000,0.781314,2.504763,6.510000,9.747765,0.054587,0.198916,39.176471,81.176471,46.740097,6.728501,12.958091,6.037573,12.031208
2013,134.488158,56.776034,1.067608,1.040000,0.778258,2.465370,7.306000,9.860076,0.054780,0.190025,39.941176,82.941176,49.573295,6.845966,13.101315,6.725407,12.785722


Средняя площадь квартиры в вводимом жилье:
<center> $AvSqVv_i= \beta_0 + \beta_1 \cdot PopAble2\_s\_MA_{i-2}+ \beta_2 \cdot PopAble3\_s\_MA_{i-2}+ \beta_3 \cdot p\_sbernas\_s_{i-1} + \varepsilon_i$, </center>

где:
- $\beta_0$ - константа, $\beta_1, \beta_2$ и $\beta_3$ - коэффициенты регрессии, $\varepsilon_i$ - случайная ошибка;
- $PopAble2\_s\_MA_{i-2}$ - доля населения, способного приобрести 1-2 комнатные квартиры в среднем за три года (лаг = 2 года);
- $PopAble3\_s\_MA_{i-2}$ - доля населения, способного приобрести 3-х комнатные квартиры в среднем за три года (лаг = 2 года);
- $p\_sbernas\_s_{i-1}$ - склонность к сбережению (лаг = 1 год);

In [46]:
iStart7_1=2010
iEnd7_1=2017
shiftPopAble2=2
shiftPopAble3=2
shiftsbernas=1

In [47]:
mod7_1 = smf.ols(formula='''AvSqVv ~ PopAble2_s_MA.shift({a1}) + PopAble3_s_MA.shift({a2}) 
                            + p_sbernas_s.shift({a3})'''.format(a1=shiftPopAble2,
                                                             a2=shiftPopAble3,
                                                             a3=shiftsbernas), 
                 data=dfzs7.loc[iStart7_1-max(shiftPopAble2,shiftPopAble3,shiftsbernas):iEnd7_1])
res7_1 = mod7_1.fit()
print(res7_1.summary())

                            OLS Regression Results                            
Dep. Variable:                 AvSqVv   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.897
Method:                 Least Squares   F-statistic:                     21.35
Date:                Mon, 22 Mar 2021   Prob (F-statistic):            0.00635
Time:                        20:10:57   Log-Likelihood:                -10.357
No. Observations:                   8   AIC:                             28.71
Df Residuals:                       4   BIC:                             29.03
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 72

/home/egor/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [48]:
dfzs7['AvSqVv_M']=res7_1.predict(dfzs7)
dfzs7.AvSqVv.fillna(dfzs7.AvSqVv_M, inplace=True)
dfzs7.loc[2031:2032,'AvSqVv']=dfzs7.loc[2030,'AvSqVv']
# dfzs7

### Модель 7.2 Средняя площадь квартиры в строящихся МКД

определяется как скользящее среднее значений средней площади квартиры в вводимом жилье ($AvSqVv$) в следующие два года 

In [49]:
dfzs7['AvSqDdy']=dfzs7['AvSqVv'].rolling(2).mean().shift(-2)

### Модель 7.4 Средняя площадь построенного ИЖД

In [50]:
dfzs7['r2DispInc_I']=dfzs7['DispInc_I'].rolling(2).apply(lambda x: np.prod(x)*100-100)
dfzs7['a2Unmpl_s']=dfzs7['Unmpl_s'].rolling(2).mean()*100

Средняя площадь построенного ИЖД:
<center> $AvSqIZD_i= \beta_0 + \beta_1 \cdot r2DispInc\_I_i+ \beta_2 \cdot a2Unmpl\_s_{i-1} + \varepsilon_i$, </center>

где:
- $\beta_0$ - константа, $\beta_1$ и $\beta_2$  - коэффициенты регрессии, $\varepsilon_i$ - случайная ошибка;
- $r2DispInc\_I_i$ - изменение реальных располагаемых доходов населения за 2 года - текущий и предшествующий <br> $r2DispInc\_I_i = DispInc\_I_i \cdot DispInc\_I_{i-1} \cdot 100 - 100$;
- $a2Unmpl\_s_{i-1}$ - уровень безработицы в среднем за 2 года - текущий и предшествующий (лаг = 1 год) $a2Unmpl\_s_{i-1} = \dfrac {Unmpl\_s_{i-1} + Unmpl\_s_{i-2}}{2} \cdot 100$;

In [51]:
iStart7_4=2009
iEnd7_4=2019
shiftDispInc=0
shiftUnmpl=1

In [52]:
mod7_4 = smf.ols(formula='''AvSqIZD ~ r2DispInc_I.shift({a1}) + a2Unmpl_s.shift({a2})'''.format(a1=shiftDispInc,
                                                                                                 a2=shiftUnmpl), 
                 data=dfzs7.loc[iStart7_4-max(shiftDispInc,shiftUnmpl):iEnd7_4])
res7_4 = mod7_4.fit()
print(res7_4.summary())

                            OLS Regression Results                            
Dep. Variable:                AvSqIZD   R-squared:                       0.317
Model:                            OLS   Adj. R-squared:                  0.147
Method:                 Least Squares   F-statistic:                     1.860
Date:                Mon, 22 Mar 2021   Prob (F-statistic):              0.217
Time:                        20:10:57   Log-Likelihood:                -27.304
No. Observations:                  11   AIC:                             60.61
Df Residuals:                       8   BIC:                             61.80
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              148.3645 

/home/egor/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [53]:
dfzs7['AvSqIZD_M']=res7_4.predict(dfzs7)
dfzs7.AvSqIZD.fillna(dfzs7.AvSqIZD_M, inplace=True)
# dfzs7

### Модель 8 Объем жилищного фонда

In [54]:
dfzs8=pdfYearZSMain[['Vvodyunits','VvodyIZDunits']].combine_first(dfzs7[['AvSqVv','AvSqIZD']]).combine_first(dfzs3[['VvodyIZD']]).combine_first(dfzs_res[['VvodyMKD']])

Располагая фактическими значениями совокупных вводов жилых единиц ($Vvodyunits$) и вводов жилых единиц ИЖД($VvodyIZDunits$), расчитывается показатель вводов жилых единиц МКД ($VvodyMKDunits$) на фактическом промежутке.

На прогнозном периоде:

$VvodyMKDunits_i= \dfrac {VvodyMKD_i}{AvSqVv_i}$

$VvodyIZDunits_i= \dfrac {VvodyIZD_i}{AvSqIZD_i}$

$Vvodyunits_i=VvodyMKDunits_i+VvodyIZDunits_i$

In [55]:
dfzs8.loc[:iLastFactYEAR,'VvodyMKDunits']=dfzs8.loc[:iLastFactYEAR,'Vvodyunits']-dfzs8.loc[:iLastFactYEAR,'VvodyIZDunits']
dfzs8.loc[iLastFactYEAR+1:,'VvodyIZDunits']=dfzs8.loc[iLastFactYEAR+1:,'VvodyIZD']/dfzs8.loc[iLastFactYEAR+1:,'AvSqIZD']
dfzs8.loc[iLastFactYEAR+1:,'VvodyMKDunits']=dfzs8.loc[iLastFactYEAR+1:,'VvodyMKD']/dfzs8.loc[iLastFactYEAR+1:,'AvSqVv']
dfzs8.loc[iLastFactYEAR+1:,'Vvodyunits']=dfzs8.loc[iLastFactYEAR+1:,'VvodyIZDunits']+dfzs8.loc[iLastFactYEAR+1:,'VvodyMKDunits']
dfzs8

,AvSqIZD,AvSqVv,VvodyIZD,VvodyIZDunits,VvodyMKD,Vvodyunits,VvodyMKDunits
2004,138.731897,68.975069,16.092900,0.116000,24.947200,0.477000,0.361000
2005,138.031496,67.268041,17.530000,0.127000,26.029500,0.515000,0.388000
2006,138.672222,65.806452,19.968800,0.144000,30.583300,0.609000,0.465000
2007,138.458421,65.601504,26.307100,0.190000,34.914200,0.722000,0.532000
2008,136.835000,64.612676,27.367000,0.200000,36.691400,0.768000,0.568000
2009,135.934000,63.821138,28.546140,0.210000,31.345420,0.702000,0.492000
2010,132.907292,62.666667,25.518200,0.192000,32.912500,0.717000,0.525000
2011,133.139303,60.683761,26.761000,0.201000,35.503600,0.786000,0.585000
2012,134.490995,59.489633,28.377600,0.211000,37.363910,0.838000,0.627000
2013,134.488158,56.776034,30.663300,0.228000,39.821600,0.929000,0.701000


### Модель 9 Объем жилищного фонда (на данный момент считается с 2019 г.) Последний факт - 2018

In [56]:
dfzs9=pdfYearZSMain[['ZhilFond','ZhilFondIZD','ZhilFondunits','ZhilFondIZDunits']].combine_first(pdfWDop[['zhiloutgor_x','zhiloutselo_x']]).combine_first(dfzs8[['VvodyMKDunits','VvodyIZDunits']]).combine_first(dfzs3[['VvodyIZD']]).combine_first(dfzs_res[['VvodyMKD']])

На фактическом промежутке рассчитывается объем жилищного фонда МКД в млн кв. м ($ZhilFondMKD$) и в млн жилых единиц ($ZhilFondMKDunits$) как разница между объемами совокупного жилищного фонда ($ZhilFond$ и $ZhilFondunits$, соответственно) и объемами жилищного фонда ИЖД ($ZhilFondIZD$ и $ZhilFondIZDunits$)

На прогнозном периоде:

$ZhilFondMKD_i = ZhilFondMKD_{i-1} + VvodyMKD_i - zhiloutgor\_x_i$
$ZhilFondMKDunits_i = ZhilFondMKDunits_{i-1} + VvodyMKDunits_i - \dfrac {zhiloutgor\_x_i}{45}$

$ZhilFondIZD_i = ZhilFondIZD_{i-1} + VvodyIZD_i - zhiloutselo\_x_i$
$ZhilFondIZDunits_i = ZhilFondIZDunits_{i-1} + VvodyIZDunits_i - \dfrac {zhiloutselo\_x_i}{80}$,

$ZhilFond_i=ZhilFondMKD_i+ZhilFondIZD_i$
$ZhilFondunits=ZhilFondMKDunits_i+ZhilFondIZDunits_i$

где:
- $zhiloutgor\_x_i$ - объем выбывающего жилья в городах;
- $zhiloutselo\_x_i$ - объем выбывающего жилья в сельской местности.

In [57]:
dfzs9.loc[:iLastFactYEAR,'ZhilFondMKD']=dfzs9.loc[:iLastFactYEAR,'ZhilFond']-dfzs9.loc[:iLastFactYEAR,'ZhilFondIZD']
dfzs9.loc[:iLastFactYEAR,'ZhilFondMKDunits']=dfzs9.loc[:iLastFactYEAR,'ZhilFondunits']-dfzs9.loc[:iLastFactYEAR,'ZhilFondIZDunits']


for i in range(iLastFactYEAR+1,iLastFORECAST+1):
    dfzs9.loc[i,'ZhilFondMKD']=dfzs9.loc[i-1,'ZhilFondMKD']+dfzs9.loc[i,'VvodyMKD']-dfzs9.loc[i,'zhiloutgor_x']
    dfzs9.loc[i,'ZhilFondIZD']=dfzs9.loc[i-1,'ZhilFondIZD']+dfzs9.loc[i,'VvodyIZD']-dfzs9.loc[i,'zhiloutselo_x']
    
    dfzs9.loc[i,'ZhilFondMKDunits']=dfzs9.loc[i-1,'ZhilFondMKDunits']+dfzs9.loc[i,'VvodyMKDunits']-dfzs9.loc[i,'zhiloutgor_x']/45
    dfzs9.loc[i,'ZhilFondIZDunits']=dfzs9.loc[i-1,'ZhilFondIZDunits']+dfzs9.loc[i,'VvodyIZDunits']-dfzs9.loc[i,'zhiloutselo_x']/80

    
dfzs9.loc[iLastFactYEAR+1:,'ZhilFond']=dfzs9.loc[iLastFactYEAR+1:,'ZhilFondMKD']+dfzs9.loc[iLastFactYEAR+1:,'ZhilFondIZD']
dfzs9.loc[iLastFactYEAR+1:,'ZhilFondunits']=dfzs9.loc[iLastFactYEAR+1:,'ZhilFondMKDunits']+dfzs9.loc[iLastFactYEAR+1:,'ZhilFondIZDunits']

dfzs9

,VvodyIZD,VvodyIZDunits,VvodyMKD,VvodyMKDunits,ZhilFond,ZhilFondIZD,ZhilFondIZDunits,ZhilFondunits,zhiloutgor_x,zhiloutselo_x,ZhilFondMKD,ZhilFondMKDunits
2004,16.092900,0.116000,24.947200,0.361000,2917.200000,794.833100,15.725665,56.854367,10.529000,4.155000,2122.366900,41.128702
2005,17.530000,0.127000,26.029500,0.388000,2955.700000,876.722600,15.670000,57.402292,22.687000,6.855000,2078.977400,41.732292
2006,19.968800,0.144000,30.583300,0.465000,3003.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,26.307100,0.190000,34.914200,0.532000,3060.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,27.367000,0.200000,36.691400,0.568000,3116.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,28.546140,0.210000,31.345420,0.492000,3176.600000,974.270500,16.261078,59.547624,8.553000,5.034000,2202.329500,43.286546
2010,25.518200,0.192000,32.912500,0.525000,3229.000000,1000.360000,16.415168,60.126706,8.934000,6.612000,2228.640000,43.711538
2011,26.761000,0.201000,35.503600,0.585000,3286.153000,1027.764400,16.595561,60.804505,8.608000,7.786500,2258.388600,44.208944
2012,28.377600,0.211000,37.363910,0.627000,3348.900000,1057.785000,16.777176,61.538125,8.282000,8.961000,2291.115000,44.760949
2013,30.663300,0.228000,39.821600,0.701000,3358.600000,1072.561900,16.783041,61.300000,6.985000,4.311000,2286.038100,44.516959


### Модель 7.3 Средняя площадь квартиры на вторичном рынке

$AvSqVtor_i=\dfrac {ZhilFondMKD_i}{ZhilFondMKDunits_i}$

In [58]:
dfzs7['AvSqVtor']=dfzs9['ZhilFondMKD']/dfzs9['ZhilFondMKDunits']

In [59]:
dfzs7['AvSqVtor']

2004    51.603061
2005    49.816995
2006          NaN
2007          NaN
2008          NaN
2009    50.877922
2010    50.985166
2011    51.084428
2012    51.185577
2013    51.352072
2014    51.468978
2015    52.215172
2016    52.532189
2017    52.167575
2018    52.151434
2019    51.766584
2020    51.817563
2021    51.870205
2022    51.946027
2023    52.024222
2024    52.120418
2025    52.221741
2026    52.319979
2027    52.413090
2028    52.495547
2029    52.563182
2030    52.613076
2031          NaN
2032          NaN
2033          NaN
2034          NaN
2035          NaN
Name: AvSqVtor, dtype: float64

### Модель 10 Модель потребности в земельных участках для строительства новых МКД (раздел считается начиная с 2017 года)

In [60]:
dfzs10=pdfWDop[['zhiloutgor_x','nowplot_x','goalplot_x','goalplotprom_x','goalplotnew_x','shareprom_s','potenprom_x','plotprom_i']].combine_first(dfzs_res[['VvodyMKD']])

<center>Всего необходимо вводов (млн кв. м): $VvodyNeeds_i = VvodyMKD_{i+2}$</center>
<br>
Необходимо вводов на повышение плотности застройки (реновация застроенных территорий)(млн кв. м):
$VNplotz_i=VNplotprom_i \cdot zhiloutgor\_x_i$

$VNplotprom_i$ - коэффициент повышения плотности застройки.<br>
$VNplotprom_i=\dfrac{goalplot\_x}{nowplot\_x}$, где 
- $goalplot\_x$ - целевая плотность застройки в двадцати крупных городских агломерациях (к 2030 г.)
- $nowplot\_x$ - текущая плотность застройки в двадцати крупных городских агломерациях (на 2017 г.)

Необходимо вводов на освоение промзон(млн кв. м):
$VNprom_i=shareprom\_s_i \cdot potenprom\_x \cdot plotprom\_i$, где 
- $shareprom\_s_i$ - доля площади застраиваемых промышленных зон в черте двадцати центров крупных городских агломераций;
- $potenprom\_x$ - потенциал строительства жилья во всех промышленных зонах в черте двадцати центров крупных городских агломераций;
- $plotprom\_i$ - коэффициент повышения плотности строительства в промышленных зонах.

Необходимо вводов на освоение новых территорий(млн кв. м):
$VNnew_i=VvodyNeeds_i - VNplotz_i - VNprom_i$

<br>
<center>Всего необходимо участков (тыс. га): $UnitsNeeds_i=UNplotz_i + UNprom_i + UNnew_i$</center>

Необходимо участков на повышение плотности застройки (реновация застроенных территорий)(тыс. га):
$UNplotz_i=\dfrac{VNplotz_i}{goalplot\_x} \cdot 1000$

Необходимо участков на освоение промзон(тыс. га):
$UNprom=\dfrac{VNprom_i}{goalplotprom\_x} \cdot 1000$, где <br>
$goalplotprom\_x$ - целевая плотность застройки в промышленных зонах городских центров двадцати крупных городских агломераций (к 2030 г.)

Необходимо участков на освоение новых территорий(тыс. га):
$UNnew=\dfrac{VNnew_i}{goalplotnew\_x} \cdot 1000$, где <br>
$goalplotnew\_x$ - целевая плотность застройки на освоенных новых территориях в двадцати крупных городских агломерациях (к 2030 г.)

In [61]:
dfzs10.loc[2017:,'VvodyNeeds']=dfzs10.loc[2017:,'VvodyMKD'].shift(-2)
dfzs10.loc[2017:,'VNplotprom']=dfzs10.loc[2030,'goalplot_x']/dfzs10.loc[2017,'nowplot_x']
dfzs10.loc[2017:,'VNplotz']=dfzs10.loc[2017:,'VNplotprom']*dfzs10.loc[2017:,'zhiloutgor_x']
dfzs10.loc[2017:,'VNprom']=dfzs10.loc[2017:,'shareprom_s']*dfzs10.loc[2030,'potenprom_x']*dfzs10.loc[2030,'plotprom_i']
dfzs10.loc[2017:,'VNnew']=dfzs10.loc[2017:,'VvodyNeeds']-dfzs10.loc[2017:,'VNplotz']-dfzs10.loc[2017:,'VNprom']

dfzs10.loc[2017:,'UNplotz']=dfzs10.loc[2017:,'VNplotz']*1000/dfzs10.loc[2030,'goalplot_x']
dfzs10.loc[2017:,'UNprom']=dfzs10.loc[2017:,'VNprom']*1000/dfzs10.loc[2030,'goalplotprom_x']
dfzs10.loc[2017:,'UNnew']=dfzs10.loc[2017:,'VNnew']*1000/dfzs10.loc[2030,'goalplotnew_x']

dfzs10.loc[2017:,'UnitsNeeds']=dfzs10.loc[2017:][['UNplotz','UNprom','UNnew']].sum(axis=1)

dfzs10

,VvodyMKD,goalplot_x,goalplotnew_x,goalplotprom_x,nowplot_x,plotprom_i,potenprom_x,shareprom_s,zhiloutgor_x,VvodyNeeds,VNplotprom,VNplotz,VNprom,VNnew,UNplotz,UNprom,UNnew,UnitsNeeds
2004,24.947200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.529000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,26.029500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.687000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,30.583300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,34.914200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,36.691400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,31.345420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.553000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,32.912500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.934000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011,35.503600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.608000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,37.363910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.282000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,39.821600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.985000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
dfBZ=dfzs2[['VvodyMKD_private','BldProcMKD','ProjectsPrivate_x']]
%store dfBZ

Stored 'dfBZ' (DataFrame)


In [63]:
dd = {'a':[1,2,3,2],'b':[3,4,5,1],'c':[2,4,6,3],'d':[3,2,2,3]}
dfdf = pd.DataFrame(data=dd)
dfdf

,a,b,c,d
0,1,3,2,3
1,2,4,4,2
2,3,5,6,2
3,2,1,3,3


In [64]:
dfdf.loc[1:,'b'].rolling(2).apply(lambda x: x.iloc[0])

1    NaN
2    4.0
3    5.0
Name: b, dtype: float64